# easYtravel: A Travel Reservations System Project


<img src="WhatsApp Image 2021-06-21 at 9.14.22 AM.jpeg" style="width:680px;height:500px"/>

### Functional Requirement

In this project, we considered the following functional requirements:

1. Welcome the user (Welcome to easYtravel.com.)
2. Location - User sees all available locations where we have presence
and user inputs a location 
3. Account Registration - User inputs First name,Last name,Mobile number, 
email address
4. Hotel Booking - user see all hotels with address available in the selected location with price range from minimum to maximum,
user selects a hotel and type of room(studio,deluxe), user inputs a check in and check out date. If room is available for those days, user proceeds to enter number of guests and the system displays the total price(rate per room * number of lodging days). User confirms booking or exits if not interested.


In [3]:
#importing libraries

import random
import json
import requests
from datetime import datetime
import pandas as pd
from IPython.display import display

This function gets the destination id in the city entered by the user.

In [4]:
def get_destination_id(city):
    '''
    This function accepts a parameter(city name) 
    and returns city names with associated 
    destination id.
    '''
    
    hotel_log = {}

    url = "https://hotels4.p.rapidapi.com/locations/search"

    querystring = {"query":{city},"locale":"en_US"}

    headers = {
    'x-rapidapi-key': "767d8882b7msh4acd2998d859b62p189d73jsndd69a7ded9e8",
    'x-rapidapi-host': "hotels4.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)

    dic = json.loads(response.text)
    city_list = dic['suggestions'][0]['entities']
     
    for i in range(len(city_list)):
        id_log = []
        for name,element in city_list[i].items():
            if name == 'destinationId':
                id_log.append(element)
            if name == 'name':
                id_log.append(element)
                hotel_log[i] = id_log
    city_dtf = pd.DataFrame(hotel_log).transpose()
    city_dtf.rename(columns={0:"destination_id", 1:"City"}, inplace=True)
    return city_dtf

We use the destination id to get in the hotel names, ratings and addresses of the hotels

In [5]:
def hotels_list(destination_id,checkOut_date,checkIn_date):
    '''
    This function returns hotel id,name and address
    given destination id, check-in and check-out dates.
    '''
    
    hotel_log = {}
    
    url = "https://hotels4.p.rapidapi.com/properties/list"

    querystring = {"adults1":"1","pageNumber":"1","destinationId":{destination_id},"pageSize":"25","checkOut":{checkOut_date},"checkIn":{checkIn_date},"sortOrder":"PRICE","locale":"en_US","currency":"USD"}

    headers = {
        'x-rapidapi-key': "767d8882b7msh4acd2998d859b62p189d73jsndd69a7ded9e8",
        'x-rapidapi-host': "hotels4.p.rapidapi.com"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    dic = json.loads(response.text)
    hotel_list = dic["data"]["body"]["searchResults"]["results"]

    for i in range(len(hotel_list)):
        hotel_details_list = []
        for name,element in hotel_list[i].items():
            if name == 'id':
                hotel_details_list.append(element)
            elif name == 'name':
                hotel_details_list.append(element)
            elif name == 'starRating':
                hotel_details_list.append(element)
            elif name == 'address':
                delem = str(element.get("streetAddress")) + "," + str(element.get("region"))
                hotel_details_list.append(delem)
                hotel_log[i] = hotel_details_list
    hotel_dtf = pd.DataFrame(hotel_log).transpose()
    hotel_dtf.rename(columns={0:"hotel_id", 1:"Hotel_Names", 2:"Rating", 3:"Address"}, inplace=True)
    return hotel_dtf

In [6]:
def get_hotel_details(id):
    
    url = "https://hotels4.p.rapidapi.com/properties/get-details"

    querystring = {"id":id}

    headers = {
    'x-rapidapi-key': "767d8882b7msh4acd2998d859b62p189d73jsndd69a7ded9e8",
    'x-rapidapi-host': "hotels4.p.rapidapi.com"
    }

    response_details = requests.request("GET", url, headers=headers, params=querystring)
    
    #hotel overview
    overview = response_details.json()['data']['body']['overview']['overviewSections']
    for view in overview:
        if 'title' in view:
            print('\033[1m' + view['title'] + '\033[0m')
            print(*view['content'], '\n', sep = '\n')

    #hygiene
    try:
        hygieneAndCleanliness = response_details.json()['data']['body']['hygieneAndCleanliness']
        print('\033[1m' + hygieneAndCleanliness['title'] + '\033[0m')
        print(*hygieneAndCleanliness['healthAndSafetyMeasures']['measures'], '\n', sep = '\n')
    except:
        print("Unavailable")
    
    #room type
    roomTypes = response_details.json()['data']['body']['propertyDescription']['roomTypeNames']
    print('\033[1m' + 'Room Types' + '\033[0m')
    print(*roomTypes, sep = '\n')
    
    #Average Price per night
    price = response_details.json()['data']['body']['propertyDescription']['featuredPrice']['currentPrice']['plain']
    print('\n\033[1m' + 'Current Average Price' + '\033[0m')
    print(f"$ {price}")

In [7]:
def booking_reg(hotel_id, first_name):
    
    #Other Personal Details
    mobile_num = input("Mobile Number (Include country code, e.g. +234): ")
    email = input("Email_Address: ")
    
    #Date
    checkin_date = str(input("Enter your checkin date (YYYY-MM-DD): "))
    checkout_date = str(input("Enter your checkout date (YYYY-MM-DD): "))
    date_format = "%Y-%m-%d"
    start = datetime.strptime(checkin_date, date_format)
    end = datetime.strptime(checkout_date, date_format)
    total_days = (end - start).days
    
    #Price
    url = "https://hotels4.p.rapidapi.com/properties/get-details"

    querystring = {"id":{hotel_id}}

    headers = {
    'x-rapidapi-key': "767d8882b7msh4acd2998d859b62p189d73jsndd69a7ded9e8",
    'x-rapidapi-host': "hotels4.p.rapidapi.com"
    }

    response_details = requests.request("GET", url, headers=headers, params=querystring)
    
    price = response_details.json()['data']['body']['propertyDescription']['featuredPrice']['currentPrice']['plain']
    total_price = price * total_days
    
    print(f'Hello {first_name}!!\nBooking cost for {total_days}days is {total_price}')

In [8]:
hotel_reserve_log = {}

def booking_confirmation(hotel_id, first_name, hotel_reserve_log):
    num = random.randint(100,600)
    booking_id = first_name[1:4] + str(num)
    print(f'Hello{first_name}!!\nHotel ID {hotel_id} successfully booked.\nYour booking ID is {booking_id}')
    hotel_reserve_log[booking_id] = {"Name": first_name, "ID":booking_id}
    return 

In [9]:
def hotel_log(log):
    
    hotel_log_dtf = pd.DataFrame(log).transpose()
    
    return hotel_log_dtf

In [10]:
def hotel_booking_system(hotel_reserve_log):
    '''
    This function is a travel reservation system that ...
    '''
    
    print("Welcome to easYtravel.com")
    runtime = 1
    while runtime:
        checker1 = input("Enter 0 for user login, 1 for Employee Login, 2 to Quit : ")
        if checker1 == "0":
            checker2 = input("Enter 0 for hotel booking, 1 to Quit : ")
            if checker2 == "0":
                des_city = str(input("Enter your desired destination (city): "))
                display(get_destination_id(des_city))
                dest_id = str(input("Enter your destination id number from the above list: "))
                checki_date = str(input("Enter your checkin date (YYYY-MM-DD): "))
                checko_date = str(input("Enter your checkout date (YYYY-MM-DD): "))
                display(hotels_list(dest_id,checko_date,checki_date))
                hotel_id = str(input("Enter your prefered hotel id number: "))
                print('\033[1m' + 'Hotel Overview' + '\033[0m')
                get_hotel_details(hotel_id)
                  
                checker3 = input("Enter 0 to proceed with hotel booking, 1 to Quit : ")
                if checker3 == "0":
                    first_name = input("First name: ")
                    last_name = input("Last name: ")
                    booking_reg(hotel_id, first_name)
                    checker4 = input("Enter 0 to confirm hotel booking, 1 to Quit : ")
                    if checker4 == "0":
                        booking_confirmation(hotel_id, first_name, hotel_reserve_log)
                    else:
                        break 
                else:
                      break
                  
        elif checker1 == "1":
            print('\033[1m' + "Hotel Log" + '\033[0m')
            display(hotel_log(hotel_reserve_log))
        elif checker1 == "2":
            break    
        else:
            hotel_booking_system()

In [ ]:
hotel_booking_system(hotel_reserve_log)

Welcome to easYtravel.com
Enter 0 for user login, 1 for Employee Login, 2 to Quit : 0
Enter 0 for hotel booking, 1 to Quit : 0
Enter your desired destination (city): Cape Town


,destination_id,City
0,1697610,Tamboerskloof
1,1613001,Cape Town
2,1665250,Cape Town City Centre
3,1697571,Camps Bay
4,10514692,Bellville


Enter your destination id number from the above list: 1613001
Enter your checkin date (YYYY-MM-DD): 2021-06-22
Enter your checkout date (YYYY-MM-DD): 2021-06-28


,hotel_id,Hotel_Names,Rating,Address
0,631395424,Long Street Backpackers,2.0,"209 Long street,Western Cape"
1,688272,Lighthouse Farm Backpackers Lodge,2.0,"A1, Oude Molen Eco Village,Western Cape"
2,649345504,Urban Hive Backpackers,2.0,"208 Long Street,Western Cape"
3,525965,Never at Home Cape Town,5.0,"107 Main Road,Western Cape"
4,568886,Mojo Hotel,3.0,"30 Regent Road,Western Cape"
5,616286,Forty8 Backpackers - Hostel,4.0,"48 Hout Street,Western Cape"
6,1412524832,Big Blue Backpackers,0.0,"7 Vesperdene Road,Western Cape"
7,549232,91 Loop,2.0,"91 Loop Street,Western Cape"
8,928477344,Zebra Crossing Backpackers,2.0,"82 New Church St,Western Cape"
9,735381760,Bedrock Guest Studios,2.5,"11 Church Street,Western Cape"


Enter your prefered hotel id number: 735381760
Hotel Overview
Main amenities
8 smoke-free guestrooms
Daily housekeeping
Breakfast available
Airport shuttle
Garden
Concierge services
Picnic area
Barbecue grills
Tour/ticket assistance
TV in a common area
Free WiFi and free parking 


Feel at home
Garden
Daily housekeeping
Barbecue grill(s)
Free self parking


What’s around
Strand Golf Club - 31 min walk
Cheetah Outreach - 43 min walk
Harmony Park - 2.5 mi / 4 km
Lwandle Migrant Labour Museum - 3.2 mi / 5.1 km
Helderberg Nature Reserve - 4.7 mi / 7.5 km
Monkey Town Primate Centre - 4.7 mi / 7.6 km
Erinvale Golf Club - 5.3 mi / 8.5 km
Bikini Beach - 5.5 mi / 8.8 km
Vergelegen Wine Estate - 6.2 mi / 10 km
Cape Floral Region Protected Areas - 6.2 mi / 10 km
Blue Rock Adventure Park - 6.4 mi / 10.4 km


COVID-19: Hygiene and cleanliness
Property is cleaned with disinfectant
Property confirms they are implementing enhanced cleaning measures


Room Types
Double Room
Sharing/ Dormitory room
Fami

,ID,Name
inn383,inn383,Winner
